In [34]:
import os
import shutil
import hashlib
from PIL import Image
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import random
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, models
from tensorflow.keras.layers import BatchNormalization
#import openai
#import json


In [35]:

# ----------------------------
# Configuration
# ----------------------------
SOURCE_DIR = "C:/Users/Ayush/ath_codee/2_custom_plant_disease_dataset__"            # path to your raw dataset
CLEANED_DIR = "C:/Users/Ayush/ath_codee/new_custom_cleaned_dataset"   # output folder for cleaned dataset
IMG_SIZE = (224, 224)                    # target size for CNN
os.makedirs(CLEANED_DIR, exist_ok=True)

# ----------------------------
# 1️⃣ Remove Corrupted Images
# ----------------------------
def remove_corrupted_images(source_dir):
    removed = 0
    for folder in os.listdir(source_dir):
        folder_path = os.path.join(source_dir, folder)
        if not os.path.isdir(folder_path):
            continue
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            try:
                img = Image.open(file_path)
                img.verify()  # verifies if image is readable
            except Exception:
                print(f"❌ Removing corrupted image: {file_path}")
                os.remove(file_path)
                removed += 1
    print(f"✅ Removed {removed} corrupted images.")

remove_corrupted_images(SOURCE_DIR)

# ----------------------------
# 2️⃣ Remove Duplicate Images
# ----------------------------
def remove_duplicate_images(source_dir):
    seen_hashes = set()
    duplicates = 0
    for folder in os.listdir(source_dir):
        folder_path = os.path.join(source_dir, folder)
        if not os.path.isdir(folder_path):
            continue
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            with open(file_path, "rb") as f:
                file_hash = hashlib.md5(f.read()).hexdigest()
            if file_hash in seen_hashes:
                print(f"🗑 Removing duplicate image: {file_path}")
                os.remove(file_path)
                duplicates += 1
            else:
                seen_hashes.add(file_hash)
    print(f"✅ Removed {duplicates} duplicate images.")

remove_duplicate_images(SOURCE_DIR)

# ----------------------------
# 3️⃣ Resize Images
# ----------------------------
def resize_images(source_dir, output_dir, target_size=(224, 224)):
    for folder in os.listdir(source_dir):
        folder_path = os.path.join(source_dir, folder)
        if not os.path.isdir(folder_path):
            continue
        output_folder = os.path.join(output_dir, folder)
        os.makedirs(output_folder, exist_ok=True)
        for file in os.listdir(folder_path):
            src_path = os.path.join(folder_path, file)
            dst_path = os.path.join(output_folder, file)
            try:
                img = Image.open(src_path).convert("RGB")
                img = img.resize(target_size)
                img.save(dst_path)
            except Exception as e:
                print(f"⚠️ Error resizing {file}: {e}")
    print(f"✅ All images resized to {target_size} and saved in '{output_dir}'.")

resize_images(SOURCE_DIR, CLEANED_DIR, IMG_SIZE)

# ----------------------------
# 4️⃣ Data Normalization (for model input)
# ----------------------------
# Note: We'll apply normalization dynamically using ImageDataGenerator below.
# This step ensures pixel values are scaled between 0 and 1.

datagen = ImageDataGenerator(rescale=1./255)
print("✅ Data normalization (rescaling 1./255) will be applied during training.")

# ----------------------------
# 5️⃣ Data Augmentation
# ----------------------------
# You’ll apply this dynamically during training for better generalization.
# Example augmentations for plant leaf datasets:

aug_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest',
    validation_split=0.15  # 85% training, 15% validation
)

# Example usage — generator creation
train_gen = aug_datagen.flow_from_directory(
    CLEANED_DIR,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = aug_datagen.flow_from_directory(
    CLEANED_DIR,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

print("✅ Data augmentation pipeline ready (rotation, flip, zoom, brightness, etc.)")


✅ Removed 0 corrupted images.
✅ Removed 0 duplicate images.
✅ All images resized to (224, 224) and saved in 'C:/Users/Ayush/ath_codee/new_custom_cleaned_dataset'.
✅ Data normalization (rescaling 1./255) will be applied during training.
Found 72 images belonging to 27 classes.
Found 0 images belonging to 27 classes.
✅ Data augmentation pipeline ready (rotation, flip, zoom, brightness, etc.)


In [36]:

# Paths
SOURCE_DIR = "C:/Users/Ayush/ath_codee/2_custom_plant_disease_dataset__"
DEST_DIR = "C:/Users/Ayush/ath_codee/new_custom_split_dataset"

# Ratios
TRAIN_SPLIT = 0.8
VAL_SPLIT = 0.1
TEST_SPLIT = 0.1

# Create destination folders
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(DEST_DIR, split), exist_ok=True)

# Split each class folder
for class_name in os.listdir(SOURCE_DIR):
    class_path = os.path.join(SOURCE_DIR, class_name)
    if not os.path.isdir(class_path):
        continue
    
    # Destination class folders
    train_folder = os.path.join(DEST_DIR, "train", class_name)
    val_folder = os.path.join(DEST_DIR, "val", class_name)
    test_folder = os.path.join(DEST_DIR, "test", class_name)
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(val_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)
    
    # Get image files
    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    random.shuffle(images)

    # Calculate split indices
    total = len(images)
    train_end = int(total * TRAIN_SPLIT)
    val_end = train_end + int(total * VAL_SPLIT)

    # Split lists
    train_imgs = images[:train_end]
    val_imgs = images[train_end:val_end]
    test_imgs = images[val_end:]

    # Copy images
    for img in train_imgs:
        shutil.copy(os.path.join(class_path, img), os.path.join(train_folder, img))
    for img in val_imgs:
        shutil.copy(os.path.join(class_path, img), os.path.join(val_folder, img))
    for img in test_imgs:
        shutil.copy(os.path.join(class_path, img), os.path.join(test_folder, img))
    
    print(f"✅ {class_name}: {len(train_imgs)} train, {len(val_imgs)} val, {len(test_imgs)} test")

print("\n🎯 Data successfully split and saved to 'custom_split_dataset/'")


✅ Aloe Leaf spot: 2 train, 0 val, 1 test
✅ Aloe Soft rot: 1 train, 0 val, 1 test
✅ Arali Leaf blight: 2 train, 0 val, 1 test
✅ Bamboo Rust: 1 train, 0 val, 1 test
✅ banana Panama wilt: 1 train, 0 val, 1 test
✅ banana Sigatoka: 2 train, 0 val, 1 test
✅ Chilli Leaf curl virus: 1 train, 0 val, 1 test
✅ Curry leaf black spot: 1 train, 0 val, 1 test
✅ Ginger Leaf spot: 2 train, 0 val, 1 test
✅ Guava Anthracnose: 0 train, 0 val, 0 test
✅ Guava Rust: 2 train, 0 val, 1 test
✅ Hibiscus Leaf spot: 1 train, 0 val, 1 test
✅ Hibiscus Rust: 3 train, 0 val, 1 test
✅ Jasmine Leaf spot: 1 train, 0 val, 1 test
✅ Jasmine Rust: 2 train, 0 val, 1 test
✅ Lemon grass Helminthosporium spot: 2 train, 0 val, 1 test
✅ Lemon grass Leaf blight: 2 train, 0 val, 1 test
✅ Lemon grass Rust: 1 train, 0 val, 1 test
✅ Mint Leaf Blight: 2 train, 0 val, 1 test
✅ Mint Rust: 3 train, 0 val, 1 test
✅ Neem Powdery mildew: 0 train, 0 val, 1 test
✅ Pepper Phytophthora foot rot: 1 train, 0 val, 1 test
✅ Powdery mildew: 4 train, 0

In [ ]:
train_dir = "C:/Users/Ayush/ath_codee/them...........DATASETS!!/2_custom_cleaned_dataset/custom_split_dataset/train"
val_dir = "C:/Users/Ayush/ath_codee/them...........DATASETS!!/2_custom_cleaned_dataset/custom_split_dataset/val"
test_dir = "C:/Users/Ayush/ath_codee/them...........DATASETS!!/2_custom_cleaned_dataset/custom_split_dataset/test"


In [37]:
train_gen = aug_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_gen = aug_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_gen = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 46 images belonging to 27 classes.
Found 0 images belonging to 27 classes.
Found 26 images belonging to 27 classes.


In [38]:
import os
import shutil
import random

def split_dataset(
    source_dir, train_dir, val_dir, test_dir,
    train_split=0.7, val_split=0.15, test_split=0.15,
    seed=42
):
    random.seed(seed)
    # Create directories if not exists
    for d in [train_dir, val_dir, test_dir]:
        if not os.path.exists(d):
            os.makedirs(d)

    classes = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]

    for cls in classes:
        cls_path = os.path.join(source_dir, cls)
        images = [f for f in os.listdir(cls_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        random.shuffle(images)

        n_total = len(images)
        n_train = int(n_total * train_split)
        n_val = int(n_total * val_split)
        n_test = n_total - n_train - n_val  # rest

        # Make class-wise directories in train/val/test
        for split_dir in [train_dir, val_dir, test_dir]:
            os.makedirs(os.path.join(split_dir, cls), exist_ok=True)

        # Copy files to respective dirs
        for i, img in enumerate(images):
            src_path = os.path.join(cls_path, img)
            if i < n_train:
                dst_path = os.path.join(train_dir, cls, img)
            elif i < (n_train + n_val):
                dst_path = os.path.join(val_dir, cls, img)
            else:
                dst_path = os.path.join(test_dir, cls, img)
            shutil.copy2(src_path, dst_path)

    print(f"Dataset split complete:")
    print(f"Train dir: {train_dir}")
    print(f"Validation dir: {val_dir}")
    print(f"Test dir: {test_dir}")

# Example usage:
source_dataset_dir = 'C:/Users/Ayush/ath_codee/2_custom_plant_disease_dataset__'
train_directory = 'path_to_train_dir'
val_directory = 'path_to_val_dir'
test_directory = 'path_to_test_dir'

split_dataset(source_dataset_dir, train_directory, val_directory, test_directory)


Dataset split complete:
Train dir: path_to_train_dir
Validation dir: path_to_val_dir
Test dir: path_to_test_dir


In [ ]:
print("Train generator class indices:", train_generator.class_indices)
print("Number of classes in train generator:", len(train_generator.class_indices))
print("Model output layer units:", model.output_shape[-1])


In [3]:
import os
import shutil
import hashlib
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# Constants
DATASET_PATH = "C:/Users/Ayush/ath_codee/2_custom_plant_disease_dataset__"
CLEANED_DATASET_PATH = "C:/Users/Ayush/ath_codee/cleaned_dataset_3"
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

# Step 1: Remove corrupted images and copy valid images to cleaned_dataset
def remove_corrupted_and_copy(src_path, dest_path):
    os.makedirs(dest_path, exist_ok=True)
    for class_folder in os.listdir(src_path):
        class_path = os.path.join(src_path, class_folder)
        if not os.path.isdir(class_path):
            continue
        dest_class_path = os.path.join(dest_path, class_folder)
        os.makedirs(dest_class_path, exist_ok=True)
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            try:
                img = Image.open(img_path)
                img.verify()  # Verify image valid
                # Reopen and convert to RGB then save resized copy
                img = Image.open(img_path).convert("RGB")
                img = img.resize(IMAGE_SIZE)
                img.save(os.path.join(dest_class_path, img_file))
            except (IOError, SyntaxError) as e:
                print(f"Removing corrupted image: {img_path}")

# Step 2: Remove duplicate images (by hash) in cleaned_dataset
def remove_duplicates(dataset_path):
    hashes = set()
    for class_folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, class_folder)
        if not os.path.isdir(folder_path):
            continue
        for img_file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_file)
            with open(img_path, "rb") as f:
                file_hash = hashlib.md5(f.read()).hexdigest()
            if file_hash in hashes:
                print(f"Duplicate found and removed: {img_path}")
                os.remove(img_path)
            else:
                hashes.add(file_hash)

# Step 6: Check class imbalance and output class counts
def print_class_distribution(dataset_path):
    class_counts = {}
    for class_folder in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_folder)
        if os.path.isdir(class_path):
            count = len([f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
            class_counts[class_folder] = count
    print("Class distribution after cleaning:")
    for cls, cnt in class_counts.items():
        print(f"{cls}: {cnt}")

# Run cleaning steps
remove_corrupted_and_copy(DATASET_PATH, CLEANED_DATASET_PATH)
remove_duplicates(CLEANED_DATASET_PATH)
print_class_distribution(CLEANED_DATASET_PATH)

# Step 4 & 5: Data normalization and augmentation with ImageDataGenerator
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    CLEANED_DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

validation_generator = train_datagen.flow_from_directory(
    CLEANED_DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=42
)

# The train_generator and validation_generator now have normalized and augmented batches ready for model training.

print(f"Number of classes: {len(train_generator.class_indices)}")
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")


Duplicate found and removed: C:/Users/Ayush/ath_codee/cleaned_dataset_3\Lemon grass Rust\3.jpg
Class distribution after cleaning:
Aloe Leaf spot: 3
Aloe Soft rot: 2
Arali Leaf blight: 3
Bamboo Rust: 2
banana Panama wilt: 2
banana Sigatoka: 3
Chilli Leaf curl virus: 2
Curry leaf black spot: 2
Ginger Leaf spot: 3
Guava Anthracnose: 0
Guava Rust: 3
Hibiscus Leaf spot: 2
Hibiscus Rust: 4
Jasmine Leaf spot: 2
Jasmine Rust: 3
Lemon grass Helminthosporium spot: 3
Lemon grass Leaf blight: 3
Lemon grass Rust: 2
Mint Leaf Blight: 3
Mint Rust: 4
Neem Powdery mildew: 1
Pepper Phytophthora foot rot: 2
Powdery mildew: 5
Rose Black spot: 2
Sooty mold: 4
Tumeric Leaf blotch: 4
Tumeric Leaf spot: 3
Found 71 images belonging to 27 classes.
Found 1 images belonging to 27 classes.
Number of classes: 27
Training samples: 71
Validation samples: 1


In [4]:
import os
import shutil
import random

# Constants
DATASET_DIR = 'C:/Users/Ayush/ath_codee/cleaned_dataset_3'  # Path to your cleaned dataset folder
TRAIN_DIR = 'C:/Users/Ayush/ath_codee/cleaned_dataset_3_train'
TEST_DIR = 'C:/Users/Ayush/ath_codee/cleaned_dataset_3_test'
TEST_SIZE = 0.15  # 15% for testing

# Create directories if they don't exist
os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(TEST_DIR, exist_ok=True)

# For each class folder in the dataset
for class_folder in os.listdir(DATASET_DIR):
    class_path = os.path.join(DATASET_DIR, class_folder)
    if not os.path.isdir(class_path):
        continue

    images = os.listdir(class_path)
    random.shuffle(images)

    # Calculate split index
    split_idx = int(len(images) * (1 - TEST_SIZE))
    train_images = images[:split_idx]
    test_images = images[split_idx:]

    # Create class folders in train and test directories
    train_class_path = os.path.join(TRAIN_DIR, class_folder)
    test_class_path = os.path.join(TEST_DIR, class_folder)
    os.makedirs(train_class_path, exist_ok=True)
    os.makedirs(test_class_path, exist_ok=True)

    # Move images to train folder
    for img in train_images:
        shutil.copy(os.path.join(class_path, img), train_class_path)

    # Move images to test folder
    for img in test_images:
        shutil.copy(os.path.join(class_path, img), test_class_path)

print("Train-test split completed and data saved.")


Train-test split completed and data saved.


In [11]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Constants
DATASET_PATH = 'C:/Users/Ayush/ath_codee/cleaned_dataset_3'
OLD_MODEL_PATH = 'C:/Users/Ayush/ath_codee/1_CNN_MODEL_code/efficientnetb0_plant_species.h5'
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)
EPOCHS = 10

# Load your existing trained model (with included pooling and classifier)
base_model = load_model(OLD_MODEL_PATH)

# Create data generators
datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

validation_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=42
)

# Get correct number of classes dynamically
num_classes = len(train_generator.class_indices)

# Remove last Dense layer from base_model and get second last layer output
x = base_model.layers[-3].output  # Dropout layer output (before Dense)

# Add new Dense layer for new number of classes
new_output = Dense(num_classes, activation='softmax')(x)

# Create new model
model = Model(inputs=base_model.input, outputs=new_output)

# Freeze base layers initially
for layer in model.layers[:-1]:
    layer.trainable = False

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS
)


Found 71 images belonging to 27 classes.
Found 1 images belonging to 27 classes.
Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.0563 - loss: 3.4461 - val_accuracy: 0.0000e+00 - val_loss: 3.3821
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 432ms/step - accuracy: 0.0986 - loss: 3.1252 - val_accuracy: 0.0000e+00 - val_loss: 3.1036
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 652ms/step - accuracy: 0.2817 - loss: 2.8682 - val_accuracy: 0.0000e+00 - val_loss: 3.0407
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 449ms/step - accuracy: 0.3803 - loss: 2.6699 - val_accuracy: 0.0000e+00 - val_loss: 3.1214
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 470ms/step - accuracy: 0.4648 - loss: 2.5379 - val_accuracy: 0.0000e+00 - val_loss: 2.9034
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 700ms/step - accuracy: 0.5775 - loss: 2.3322 - val_accuracy: 1.0000 - val_loss: 2.7180
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 769ms/step - accuracy: 0.6901 - loss: 2.1446 - val_accuracy: 0.0000e+00 - val_loss: 2.6041
Epoch 8/10
3/3 

In [14]:
model.save('new_efficientnetb0_disease_detector.keras')
print("Model saved as new_efficientnetb0_disease_detector.keras")


Model saved as new_efficientnetb0_disease_detector.keras


In [15]:
import tensorflow as tf

# Replace the path below with your actual model file path
model_path = 'new_efficientnetb0_disease_detector.keras'  # or .h5 if using legacy format

# Load the model
model = tf.keras.models.load_model(model_path)

# Print model summary
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 27)             │        34,587 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,153,334 (15.84 MB)

 Trainable params: 34,587 (135.11 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

 Optimizer params: 69,176 (270.22 KB)